In [6]:
import pandas as pd
import os
import numpy as np
import trimesh

- Curvature estimation (Gaussian curvature) was tested but removed. 
 
- Reason: curvature metrics were unreliable for STL models with gaps, degenerate faces, or poor mesh quality which produced errors for noisy or incomplete geometry.

In [ ]:
# Simplified curvature estimation that's more robust to degenerate faces
def estimate_gaussian_curvature_robust(mesh):
    try:
        # Try to use trimesh's built-in vertex_faces if available
        vertex_faces = mesh.vertex_faces
        curvature = np.zeros(len(mesh.vertices))
        
        for i in range(len(mesh.vertices)):
            try:
                faces = vertex_faces[i]
                valid_faces = faces[faces != -1]
                
                if len(valid_faces) == 0:
                    curvature[i] = 0
                    continue
                    
                angles = []
                vertex = mesh.vertices[i]
                
                for face_idx in valid_faces:
                    face = mesh.faces[face_idx]
                    # Find position of current vertex in the face
                    vertex_pos = np.where(face == i)[0]
                    if len(vertex_pos) == 0:
                        continue
                        
                    # Get the other two vertices
                    other_indices = face[face != i]
                    if len(other_indices) >= 2:
                        v1, v2 = mesh.vertices[other_indices[:2]]
                        
                        # Calculate vectors from current vertex to others
                        vec1 = v1 - vertex
                        vec2 = v2 - vertex
                        
                        # Calculate angle
                        norm1, norm2 = np.linalg.norm(vec1), np.linalg.norm(vec2)
                        if norm1 > 1e-8 and norm2 > 1e-8:
                            cos_angle = np.clip(np.dot(vec1, vec2) / (norm1 * norm2), -1.0, 1.0)
                            angle = np.arccos(cos_angle)
                            angles.append(angle)
                
                # Calculate curvature using angle deficit
                if len(angles) > 0:
                    angle_sum = np.sum(angles)
                    curvature[i] = (2 * np.pi) - angle_sum
                else:
                    curvature[i] = 0
                    
            except Exception:
                curvature[i] = 0
                
        return curvature
        
    except Exception:
        # Fallback: use a simple geometric measure
        print("Warning: Falling back to simplified curvature estimation")
        return np.zeros(len(mesh.vertices))
# issue was that some models with blannk faces were causing errors

- Implemented `simple_flatness_analysis`func where it uses height ratio, Z-variation, normal variation, and aspect ratio for flatness detection.
- Reason: Avoided curvature due to errors on degenerate meshes; improved reliability for noisy STL files. Aspect ratio helps distinguish flat, non-elongated shapes from thin, elongated ones.

In [ ]:
# Alternative simple flatness estimation without vertex_faces
def simple_flatness_analysis(mesh):
    try:
        # Basic geometric metrics
        bounds = mesh.bounds
        dimensions = bounds[1] - bounds[0]
        height_ratio = dimensions[2] / max(dimensions[0], dimensions[1])
        
        # Z-coordinate variation
        z_coords = mesh.vertices[:, 2]
        z_std = np.std(z_coords)
        z_range = np.max(z_coords) - np.min(z_coords)
        z_std_normalized = z_std / z_range if z_range > 0 else 0
        
        # Surface normal variation (simplified)
        try:
            face_normals = mesh.face_normals
            # Calculate how much normals vary (flatness indicator)
            normal_std = np.std(face_normals, axis=0)
            normal_variation = np.mean(normal_std)
        except Exception:
            normal_variation = 0.5  # Default moderate variation
        
        return {
            'height_ratio': height_ratio,
            'z_std_normalized': z_std_normalized,
            'normal_variation': normal_variation,
            'dimensions': dimensions
        }
    except Exception as e:
        raise e

# NEW (fast): quantify how much of the mesh area lies on a few dominant plane orientations
# Vectorized spherical binning of face normals (treat n and -n as same) to avoid O(F^2) clustering
# bin_degrees controls angular bin size; larger bins -> faster and more tolerant grouping


- Added `planarity_dominance` func to quantify dominant plane orientation using spherical binning of face normals.
- Reason: Fast, robust way to detect planar surfaces and distinguish flat vs. free-form models.

In [ ]:

def planarity_dominance(mesh, bin_degrees: float = 15.0):
    """
    Fast area-weighted dominance of face-normal orientation.
    Returns fraction of total surface area in the most-populated normal bin.
    Opposite directions are treated as identical (hemisphere mapping).
    """
    try:
        normals = mesh.face_normals
        if normals is None or len(normals) == 0:
            return 0.0
        areas = getattr(mesh, 'area_faces', None)
        if areas is None:
            areas = np.ones(len(normals), dtype=np.float64)
        else:
            areas = areas.astype(np.float64, copy=False)
        # Normalize to unit and map to canonical hemisphere so n and -n coincide
        norms = np.linalg.norm(normals, axis=1)
        normals = normals / (norms[:, None] + 1e-12)
        mask = (normals[:, 0] < 0) | ((normals[:, 0] == 0) & (normals[:, 1] < 0)) | ((normals[:, 0] == 0) & (normals[:, 1] == 0) & (normals[:, 2] < 0))
        normals[mask] = -normals[mask]
        # Spherical coordinates
        theta = np.arccos(np.clip(normals[:, 2], -1.0, 1.0))   # 0..pi
        phi = np.arctan2(normals[:, 1], normals[:, 0])        # -pi..pi
        phi = np.where(phi < 0, phi + 2*np.pi, phi)           # 0..2pi
        d = np.deg2rad(bin_degrees)
        nbins_theta = max(1, int(np.ceil(np.pi / d)))
        nbins_phi = max(1, int(np.ceil(2*np.pi / d)))
        btheta = np.minimum((theta / d).astype(np.int32), nbins_theta - 1)
        bphi = np.minimum((phi / d).astype(np.int32), nbins_phi - 1)
        keys = (btheta.astype(np.int64) * nbins_phi) + bphi.astype(np.int64)
        # Sum areas per bin using sorting + reduceat (vectorized)
        order = np.argsort(keys)
        keys_sorted = keys[order]
        areas_sorted = areas[order]
        uniq, start_idx, counts = np.unique(keys_sorted, return_index=True, return_counts=True)
        bin_sums = np.add.reduceat(areas_sorted, start_idx)
        total_area = float(areas_sorted.sum()) + 1e-12
        return float(bin_sums.max()) / total_area if bin_sums.size else 0.0
    except Exception:
        return 0.0


Test for a single STL file from the dataset

In [ ]:
data_dir = r'C:\\Users\\Anuhas\\Documents\\Research\\Thingiverse\\data'
stl_files = [f for f in os.listdir(data_dir) if f.endswith('.stl')]

if stl_files:
    # Test with the first available STL file
    test_file = stl_files[0]
    file_path = os.path.join(data_dir, test_file)
    
    print(f"Analyzing: {test_file}")
    
    try:
        mesh = trimesh.load(file_path)
        
        # Use simple analysis first
        simple_analysis = simple_flatness_analysis(mesh)
        print(f"Height ratio: {simple_analysis['height_ratio']:.4f}")
        print(f"Z-variation (normalized): {simple_analysis['z_std_normalized']:.4f}")
        print(f"Normal variation: {simple_analysis['normal_variation']:.4f}")
        # Show new planarity metric
        pd_score = planarity_dominance(mesh)
        print(f"Planarity dominance: {pd_score:.3f}")
        
        print(f"Dimensions: width={simple_analysis['dimensions'][0]:.2f}, depth={simple_analysis['dimensions'][1]:.2f}, height={simple_analysis['dimensions'][2]:.2f}")
        
        # Check criteria
        crit = {
            'planarity_dominant': pd_score >= 0.70,
            'height_ratio_flat': simple_analysis['height_ratio'] < 0.20,
            'z_variation_flat': simple_analysis['z_std_normalized'] < 0.35,
            'normal_variation_flat': simple_analysis['normal_variation'] < 0.35
        }
        votes = sum(crit.values())
        pred = 'FLAT' if (crit['planarity_dominant'] and (crit['z_variation_flat'] or crit['normal_variation_flat'])) or votes >= (len(crit)/2) else 'FREE-FORM'
        print(f"Criteria votes: {votes}/{len(crit)}")
        print(f"Prediction: {pred}")
        
        # Try curvature analysis with robust method
        try:
            curvature = estimate_gaussian_curvature_robust(mesh)
            mean_curvature = np.mean(np.abs(curvature))
            print(f"Mean Gaussian curvature: {mean_curvature:.6f}")
        except Exception as e:
            print(f"Curvature analysis failed: {e}")
            mean_curvature = 0
        
        # Simple classification
        is_flat = (simple_analysis['height_ratio'] < 0.15 and 
                  simple_analysis['z_std_normalized'] < 0.3)
        print(f"Simple prediction: {'FLAT' if is_flat else 'FREE-FORM'}")
        
    except Exception as e:
        print(f"Error loading {test_file}: {e}")
else:
    print("No STL files found in data directory.")

Analyzing: 103742.stl
Height ratio: 0.0185
Z-variation (normalized): 0.3536
Normal variation: 0.5685
Planarity dominance: 0.247
Dimensions: width=135.20, depth=99.00, height=2.49
Criteria votes: 1/4
Prediction: FREE-FORM
Mean Gaussian curvature: 0.485749
Simple prediction: FREE-FORM
Mean Gaussian curvature: 0.485749
Simple prediction: FREE-FORM


Test for all STL files in the dataset and save results to CSV


In [ ]:
from datetime import datetime

# Get all STL files from the data directory
data_dir = r'C:\\Users\\Anuhas\\Documents\\Research\\Thingiverse\\data'
stl_files = [f for f in os.listdir(data_dir) if f.endswith('.stl')]

print(f"Found {len(stl_files)} STL files to analyze...")

# Store results
results = []
processed = 0
errors = 0

# Ensure comprehensive_stl_analysis exists (define minimal wrapper if missing)
if 'comprehensive_stl_analysis' not in globals():
    def comprehensive_stl_analysis(file_path):
        mesh = trimesh.load(file_path)
        sm = simple_flatness_analysis(mesh)
        try:
            pd_score = planarity_dominance(mesh)
        except Exception:
            pd_score = 0.0
        width, depth, height = sm['dimensions']
        aspect_ratio_1 = height / width if width > 0 else 0
        aspect_ratio_2 = height / depth if depth > 0 else 0
        max_ar = max(aspect_ratio_1, aspect_ratio_2)
        crit = {
            'planarity_dominant': pd_score >= 0.70,
            'height_ratio_flat': sm['height_ratio'] < 0.20,
            'z_variation_flat': sm['z_std_normalized'] < 0.35,
            'normal_variation_flat': sm['normal_variation'] < 0.35,
            'aspect_ratio_flat': max_ar < 0.30
        }
        votes = sum(crit.values())
        pred = 'FLAT' if (crit['planarity_dominant'] and (crit['z_variation_flat'] or crit['normal_variation_flat'])) or votes >= (len(crit)/2) else 'FREE-FORM'
        return {
            'filename': os.path.basename(file_path),
            'prediction': pred,
            'height_ratio': sm['height_ratio'],
            'z_std_normalized': sm['z_std_normalized'],
            'normal_variation': sm['normal_variation'],
            'planarity_dominance': pd_score,
            'max_aspect_ratio': max_ar,
            'criteria_votes': votes,
            'total_criteria': len(crit),
            'dimensions': sm['dimensions'].tolist() if hasattr(sm['dimensions'], 'tolist') else list(sm['dimensions'])
        }

for filename in stl_files:
    file_path = os.path.join(data_dir, filename)
    print(f"Processing {processed + 1}/{len(stl_files)}: {filename}")
    
    try:
        result = comprehensive_stl_analysis(file_path)
        if result['prediction'] == 'ERROR':
            errors += 1
            print(f"  ERROR: {result.get('error', 'Unknown error')}")
        else:
            print(f"  Prediction: {result['prediction']} (votes: {result['criteria_votes']}/{result['total_criteria']}, planarity: {result.get('planarity_dominance', 0):.2f})")
        results.append(result)
    except Exception as e:
        errors += 1
        print(f"  ERROR (pipeline): {e}")
    processed += 1

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Save to CSV with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f'stl_analysis_results_{timestamp}.csv'
output_path = os.path.join(r'C:\\Users\\Anuhas\\Documents\\Research\\Thingiverse', output_file)

df_results.to_csv(output_path, index=False)

print(f"\n=== ANALYSIS COMPLETE ===")
print(f"Total files processed: {processed}")
print(f"Successful analyses: {processed - errors}")
print(f"Errors: {errors}")
print(f"Results saved to: {output_path}")

# Display summary statistics
if processed > errors:
    predictions = df_results[df_results['prediction'] != 'ERROR']['prediction']
    print(f"\n=== PREDICTION SUMMARY ===")
    print(f"FLAT models: {sum(predictions == 'FLAT')}")
    print(f"FREE-FORM models: {sum(predictions == 'FREE-FORM')}")
    
    # Show first few results (updated column names)
    print(f"\n=== FIRST 5 RESULTS ===")
    display_cols = ['filename', 'prediction', 'planarity_dominance', 'height_ratio', 'z_std_normalized', 'normal_variation', 'max_aspect_ratio']
    print(df_results[display_cols].head())
    
    # Show available columns for reference
    print(f"\n=== ALL AVAILABLE COLUMNS ===")
    print(list(df_results.columns))

Found 249 STL files to analyze...
Processing 1/249: 103742.stl
  Prediction: FREE-FORM (votes: 1/4, planarity: 0.25)
Processing 2/249: 103815.stl
  Prediction: FLAT (votes: 2/4, planarity: 0.11)
Processing 3/249: 103817.stl
  Prediction: FLAT (votes: 2/4, planarity: 0.11)
Processing 3/249: 103817.stl


Found 249 STL files to analyze...
Processing 1/249: 103742.stl
  Prediction: FREE-FORM (votes: 1/4, planarity: 0.25)
Processing 2/249: 103815.stl
  Prediction: FLAT (votes: 2/4, planarity: 0.11)
Processing 3/249: 103817.stl
  Prediction: FLAT (votes: 2/4, planarity: 0.11)
Processing 3/249: 103817.stl


  Prediction: FREE-FORM (votes: 0/4, planarity: 0.10)
Processing 4/249: 103821.stl
  Prediction: FREE-FORM (votes: 1/4, planarity: 0.32)
Processing 5/249: 103824.stl
  Prediction: FREE-FORM (votes: 0/4, planarity: 0.46)
Processing 6/249: 103825.stl

Processing 4/249: 103821.stl
  Prediction: FREE-FORM (votes: 1/4, planarity: 0.32)
Processing 5/249: 103824.stl
  Prediction: FREE-FORM (votes: 0/4, planarity: 0.46)
Processing 6/249: 103825.stl
  Prediction: FREE-FORM (votes: 0/4, planarity: 0.46)
Processing 7/249: 103826.stl
  Prediction: FREE-FORM (votes: 0/4, planarity: 0.46)
Processing 8/249: 104187.stl
  Prediction: FREE-FORM (votes: 0/4, planarity: 0.46)
Processing 7/249: 103826.stl
  Prediction: FREE-FORM (votes: 0/4, planarity: 0.46)
Processing 8/249: 104187.stl
  Prediction: FLAT (votes: 2/4, planarity: 0.32)
Processing 9/249: 104188.stl
  Prediction: FLAT (votes: 2/4, planarity: 0.33)
Processing 10/249: 104290.stl
  Prediction: FLAT (votes: 2/4, planarity: 0.32)
Processing 9/249: